# Polution Time-Series Forecasting with LSTM
The supervised pollution learning problem: predict the pollution at the current time $𝑡$ given the pollution measurement and weather conditions at the prior time steps $(𝑡−1), \ldots, (t-i)$. WE are going to model here the simplest version when $i=1$, i.e. predict from previous time step.<br> 
As usual we start with loading necessary packages

In [ ]:
import tensorflow as tf
from pandas import read_csv
from datetime import datetime
import os
import matplotlib.pyplot as plt
from pandas import DataFrame
from pandas import concat
from keras import models
from keras import layers
from keras import optimizers

## Polution Data
The data set is taken from UCI data set repository: https://archive.ics.uci.edu/ml/machine-learning-databases/00381/  Click on data folder to download. I renamed it raw_pollution.csv.<br><br>
Weather conditions are given by combination of feature values measured at a given date/time. We assume that we do not know all important features, so there is some uncertainty involved and it is encoded in the current unknow state of the weather system defined by undetermined  weather  conditions. <br>
The data in the file includes the date-time, the pollution level called PM2.5 concentration, and the weather information. Table columns are:
<ul>
    <li>index column</li>
    <li> year: year of data in this column</li>
    <li>month: month of data in this colum</li>
    <li>day: day of data in this column</li>
    <li>hour: hour of data in this column</li>
    <li>pm2.5: PM2.5 concentration column. This is the target column</li>
    <li>DEWP: Dew Point column</li>
    <li>TEMP: temperature column</li>
    <li>PRES: pressure column</li>
    <li>cbwd: Combined wind direction column</li>
    <li>Iws: Cumulated wind speed column</li>
    <li>Is: Cumulated hours of snow column</li>
    <li>Ir: Cumulated hours of rain</li>
</ul>
When rpocessing we would need to convert "year","month","day", "hour" columns into $daytime$ Python object. For that I define parse function that is nothing more than a call to <b>strptime()</b> standard Python function. It takes takes two arguments:
<ul>
    <li>string (that be converted to datetime)</li>
    <li>format code</li>
</ul>
Based on the string and format code used, the method returns its equivalent datetime object. For example, if the call is

    - datetime.strptime(x, '%Y %m %d %H')
 then the format of argiment $x$ that is fed to the call must be "year","month","day", "hour", as for example in "2010 1 5 17".

In [ ]:
def parse(x):
    return datetime.strptime(x, '%Y %m %d %H')

## Input data
As usual check if we are in Google colab or working locally

In [ ]:
try:
    from google.colab import drive
    IN_COLAB=True
except:
    IN_COLAB=False

Set the path to the directory into which the original dataset is located. New Python twist: I assume that data is located in a subdirectory of a directory from which we launched the program. So I take its name using "os.getcwd".<br>
The subdirectory in which I assume the data is located is ../Data/pollution when working locally and in ../My Drive/courses/Deep Learning/data/pollution when on Colab. If there is no data file in this subdirectory, then I exit.<br>
My original file is named "raw_pollution.csv". But I do not want to wreck an original file as I'll be preprocessing data, so I'll preprocess it into a new file "pollution.csv" in the next cell if I have not done it before.

In [ ]:
if not IN_COLAB:
#    dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
    dir_path = os.getcwd()
    data_dir = os.path.join(dir_path, 'pollution')
else:
    drive.mount('/content/gdrive')
    dir_path = os.path.dirname(os.path.realpath(os.path.abspath('')))
    data_dir = os.path.join(dir_path, '/content/gdrive/My Drive/courses/Deep Learning/data/pollution')
if not os.path.exists(data_dir):
    exit(3)
copy=False
dst=os.path.join(data_dir,'pollution.csv')

Next, if i have not created processed file "pollution.csv" before I read the original file "raw_pollution.csv" and process it and write out processed file  "pollution.csv". If I already processed raw file then I just read it.<br>
How do I process raw file if I need it? I extensively use "pandas" module: 
<ol>
    <li> I read the raw data file into "dataset" data-frame variable. I use "date_parser" function of pandas "read_csv" module to create a column with datetime object that replaces 4 columns - "year","month","day", "hour". Data_parser converts a sequence of string columns to an array of datetime instances. The default uses dateutil.parser.parser to do the conversion but I use my own parse function. Read_csv passes value of "parse_dates" argument concatenating (row-wise) the string values from the columns defined by parse_dates into a single array and pass that to the parser. The resulting column becomes index column of the dataframe</li>
    <li> I use pandas dataframe.drop to drop index column (name is 'No', axis=1 means columns, inplaceTrue means do not make a copy - just do it on the frame)</li>
    <li> I define new names of the columns of dataframe  replacing "pm2.5  DEWP  TEMP    PRES cbwd    Iws  Is  Ir" with  "pollution  dew  temp   press wnd_dir  wnd_spd  snow  rain" note that name replacement can be applied to a list of all coumns except for index column which can only be renamed explicitly.</li>
    <li> which is what is done next - index column is named "date">/li>
     <li> I replace 'NA' in pollution column with 0 and drop first 24 hours of observations as you can see (look into raw file) it has no pollution data there</li>
    <li>save dataset using pandas to_csv method.</li>
</ol>
        

In [ ]:
def parse(x):
    return datetime.strptime(x, '%Y %m %d %H')

In [ ]:
if not os.path.exists(dst):
    copy=True
    src = os.path.join(data_dir,'raw_pollution.csv')
    dataset = read_csv(src,  parse_dates = [['year', 'month', 'day', 'hour']], index_col=0, date_parser=parse)
    dataset.drop('No', axis=1, inplace=True)
    dataset.columns = ['pollution', 'dew', 'temp', 'press', 'wnd_dir', 'wnd_spd', 'snow', 'rain']
    dataset.index.name = 'date'
    dataset['pollution'].fillna(0, inplace=True)
    dataset = dataset[24:]
    dataset.to_csv(dst)
    print(dataset[:5])
if not copy:
    dataset = read_csv(dst, header=0, index_col=0)

I then plot all columns. To do so I need to conver a dataframe to a 2:d array, i.e. get rid of all data frame things such as column names, axises, index column designations, etc. This is what datafra.values method does. And then plotting of the data

In [ ]:
values = dataset.values
# specify columns to plot
groups=list(range(0,7+1))
i = 1
# plot each column
plt.figure()
for group in groups:
    plt.subplot(len(groups), 1, i)
    plt.plot(values[:, group])
    plt.title(dataset.columns[group], y=0.5, loc='right')
    i += 1
plt.show()

## Normalize, reformat and prepare data for learning
<ul>
    <li> Import from sklearn label encoder and scaler</li>
    <li> Using sklearn.preprocessing.LabelEncoder.fit_transform method take categorical values of the column 4 (wind) and convert these values to integers by enumerating the categorical values of the column </li>
    <li> Convert all integers to real values for subsequent scaling</li>
    <li>Using sklearn.preprocessing.MinMaxScaler.fit_transform method take all columns and uniformly scale values in columns between 0 and 1</li>
</ul>

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

### series_to_supervised() Function
Next we convert data to format that is required to use LSTM models. This is done using series_to_supervised function borrowed from https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/ (this is not my function, but it is free to use as allowed by the author). <br>
Technically, when we are forecusting time series (or any sequence) at the current time (position) $𝑡$ for the future times (positions) $(𝑡+1, \ldots, 𝑡+𝑛)$ we are changing time origin: from the forecast times in future and to predicting from the past observations, i.e. we are taking as current the time as the time in which we want the prediction, so $t^{'}=t+n$, and we are computing it from backward values, i.e. we are computing value at $t^{'}$ from values $(𝑡^{'}−1, 𝑡^{'}−𝑛)$ that are used to make forecasts. Thus to make this view change we (i.e. time series data into a format on which we can apply supervised learning) we use the Pandas shift() function. The DataFrame.shift() create copies of columns pushed forward (rows of $NaN$ values added to the front) or pulled back (rows of $NaN$ values added to the end) a number of time units that are needed for prediction. This number is the argument of DataFrame.shift(). This function can be used to create new columns (features) of lag observations and the column of forecast observations for target variable necessary for supervised learning. Here is mock example. Suppose we need to predict a series of a single variable that only deneds on its own previous $2$ values and we wnat to predict next $2$ values for a currnt time. Suppose we have the following $9$ observations. Then for our prediction we'll need to have $14 \times 5$ dataframe on which we'll be learning. Then we'll need to drop all rows that conatin "na". So the transformation is <br>
<center>
$\begin{array}{l} 2\\ 3\\ 4\\ 3\\ 4\\ 5\\ 4 \\ 4\\ 5 \end{array}\ \ \ $ becomes $\ \ \ \begin{array}{|l|l|l|l|l|} 2 & na & na &na & na \\ \hline 3 & 2 & na & na & na\\ \hline 4 & 3 & 2 & na & na \\ \hline 3 & 4 & 3 & 3 & na\\ \hline 4 & 3 & 4 & 3 & 2 \\ \hline 5 & 4 & 3 & 4 & 3\\ \hline 4 & 5 & 4 & 3 & 4\\ \hline 4 & 4 & 5 & 4 & 4 \\ \hline 5 & 4 & 4 & 5 & 4\\ \hline na & 5 & 4 & 4 & 5\\ \hline na & na &5 & 4 & 4 \\ \hline na & na & na & 5 & 4 \\ \hline na & na & na & na & 5 \end{array}\ \ \ $ becomes $\ \ \ \begin{array}{|l|l|l|l|l|} 4 & 3 & 4 & 3 & 2 \\ \hline 5 & 4 & 3 & 4 & 3\\ \hline 4 & 5 & 4 & 3 & 4\\ \hline 4 & 4 & 5 & 4 & 4 \\ \hline 5 & 4 & 4 & 5 & 4 \end{array}\ \ \ $</center>
<br> 
Arguments of series_to_supervised are:
<ul>
    <li> data: Sequence of observations as a list or 2D NumPy array.</li>
    <li>n_in: Number of lag observations as input (𝑋). Values may be between [1..len(data)].</li>
    <li> n_out: Number of observations as output (𝑌). Values may be between [0..len(data)-1].</li>
    <li> dropnan: Boolean whether or not to drop rows with NaN values.</li>
</ul>
The function returns It returns Pandas DataFrame of series framed for supervised learning: if we had $𝑛\times 𝑝$ data frame with 𝑝  features and 𝑛 time slices then if we want to have new data dependent on $𝑘$ previous time values $𝑡−1,\ldots, 𝑡−𝑘$ and we are predicting $𝑠$ new time values at times $𝑡,\ldots,𝑡+𝑠−1$ then the new data frame has dimensions $(𝑛\times p)\cdot (𝑘+𝑠+1)$. 
<br>
Essentially this function
<ol>
    <li>Shifts columns forward n_in times to create $𝑡−𝑖$ columns and n-out times backward from $t$ to create $𝑡+𝑖$ feature vectors</li>
    <li>Each time a new frame of feature vectors shifted by time $𝑡\pm 𝑖$ a set of $𝑝$ columns is created. It is treated as a set of additional features i.e. it is merged with the previous data frame adding to it $𝑝$ new features</li>
    <li>It starts with checking if it is one dimensional or multidimensional variable prediction. In both cases define n_var number of dimensions/variables (either 1 or # of columns)</li>
 <li> If it is multidimensional then convert numpy.ndarray to pandas DataFrame and create empty lists where we’ll put in data</li>
</ol>

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1] # #-of-vars is either 1 if one dim sequence or number of cols
    df = DataFrame(data) #convert from array of matrix to data frame
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))       #shift all frame columns by i (1 )up obtaining t-i (t-1) values
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)] #form names of vars from 1 to 8 at time t-i
        # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))   # put frame back in place
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        return agg

Since I am only doing $t+1$ prediction we need number of lag observations for input to be 1 and number of observations to output be 1. Drop columns in 𝑡+1 that are not target variables (not pollution)from added frames. Pandas dataframe.drop method allows to dropSinc we reproduced/concatenated all columns (axis is 1 so it is indeed columns), and we do not need for pprediction all feature columns at time $t+1$ so we earse these columns. Note that columns at time $t+1$ aren't part of the model only feature columns of t-1 are. But if we were predicting the dependence on 2 time slots back then features of $t+1$ would be needed. Moreover I treat previous polution value as a feature on which next polluiton value depends. But if we were not using it as a feature we would have had to drop it.

In [ ]:
reframed = series_to_supervised(scaled, 1, 1)
reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
print(reframed.head())

### Prepare Data
By using dataframe.values method only the values in the DataFrame will be returned, the axes labels will be removed. Could use to_numpy instead. Then I split index number into 3 groups: train, validation and test sets of index numbers. Using each index group respectively create training data, validation data and testing data. Split data for each group into independent feature variables and target variable.

In [ ]:
values = reframed.values
n_train_hours = int(round(len(values)/3))
n_valid_hours=int(round(2*len(values)/3))
train = values[:n_train_hours, :]
valid=values[n_train_hours:n_valid_hours, :]
test = values[n_valid_hours:, :]
# split into training input, validation output and later testing input data
train_X, train_y = train[:, :-1], train[:, -1]
valid_X, valid_y = valid[:, :-1], valid[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
valid_X = valid_X.reshape((valid_X.shape[0], 1, valid_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, valid_X.shape, valid_y.shape, test_X.shape, test_y.shape)

## Define and fit the model
A do standard vanilla LSTM is an LSTM model that has a single hidden layer of LSTM units, and an output layer used to make a prediction. I define the LSTM layer with 50 LSTM neurons in the only hidden layer and 1 dense/linear neuron in the output layer for predicting pollution. Multiple hidden LSTM layers can be stacked one on top of another in what is referred to as a Stacked LSTM model. It is necessary when we want to discover complex time dependencies just like in convolutional nets. We do not need it here. Why 50 neurons? it is a practical matter because it is bit enough number to carry memmory of previous steps and small enough to not have vanishing gradiaent problem. 
<ul>
    <li>The input shape has 1 time step with 8 features</li>.
    <li>Optimizer is Adam version of stochastic gradient descent. Works well for LSTMs - later if wqe have time</li>
    <li> Loss is Mean Absolute Error (MAE)</li>
    <li>Batch size 96 (hours = 4 days)</li>
</ul>
Why did I choose MAE loss?The beauty of the MAE is that since we are taking the absolute value, all of the errors will be weighted on the same linear scale. Thus, we won’t be putting too much weight on our outliers as MSE function does. This loss evens out errors of how well our model is performing which is good for a volatile target such as pollution.
<br>
The standard measures of quality of the model for time series are:
<ul>
    <li> Mean absolute error $\frac{1}{n}\sum_{i=1}^n|y_i -\hat{y}_i|$ </li>
    <li> Mean squared error (MSE) $\frac{1}{n}\sum_{i=1}^n(y_i -\hat{y}_i)^2$</li>
    <li> Root MSE (RMSE) $\sqrt{\frac{1}{n}\sum_{i=1}^n(y_i -\hat{y}_i)^2}$</li>
</ul>
I choose to watch mean squared error exactly to see how much my model evens out. Am I missing that my model is underfiting which I didn't noitce in losse because of evening all errors out.

In [ ]:
model = models.Sequential()
model.add(layers.LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(layers.Dense(1))
model.compile(loss='mae', optimizer='adam', metrics=['mse'])
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=96, validation_data=(valid_X, valid_y), verbose=2, shuffle=False)

Now plot training

In [ ]:
plt.close()
plt.plot(history.history['mse'], 'r',label='MSE')
plt.plot(history.history['val_mse'], 'b', label='Validation MSE')
plt.title('Training and validation Mean Squared Error')
plt.legend()
plt.figure()
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='valid')
plt.title('Training and validation Loss (MAE)')
plt.legend()
plt.show()

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
from numpy import concatenate

yhat = model.predict(test_X)
rmse0 = sqrt(mean_squared_error(test_y, yhat))
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE scaled: %.3f' % rmse0)
print('Test RMSE absolute: %.3f' % rmse)

## Additional reading
Pollution modeling: https://luisdamiano.github.io/work/gsoc17_iohmm_financial_time_series.html#:~:text=Code%202017%20program.-,1%20The%20Input%2DOutput%20Hidden%20Markov%20Model,control%20signal%2C%20to%20output%20sequences
https://towardsdatascience.com/forecasting-air-pollution-with-recurrent-neural-networks-ffb095763a5c
Chollet, https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/6.3-advanced-usage-of-recurrent-neural-networks.ipynb 